# 上证指数涨跌预测

Install required moduels      

    pip install numpy
    pip install scipy
    pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation

c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import peforth
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
data=pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

In [23]:
# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

In [ ]:
data.index

In [ ]:
data.query('index < "1997-06-20"')

In [24]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature));

In [ ]:
peforth.push(data).dictate('constant data');
peforth.push(dayfeature).dictate('constant dayfeature');
peforth.push(x).dictate('value x');
peforth.push(y).dictate('value y');

In [ ]:
%f data :> shape . cr
%f data :> shape[0] . cr
%f data :> shape[0]-v('dayfeature') . cr
peforth.ok();

In [25]:
# 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

for i in range(0,data.shape[0]-dayfeature):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

In [27]:
'''
data.shape[0]-dayfeature 是 4602 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4602, 751)
dayfeature 是 150 
他用下一天，第 151 天，的「開盤」vs 當天的「收盤」來當作這天是否有漲。這點怪怪的，應該用「第 151 天」
的「收盤」來跟「第 150 天」的「收盤」比較才對。
'''

for i in range(0,data.shape[0]-dayfeature):
    if data.iloc[i+dayfeature][u'收盘价']>=data.iloc[i+dayfeature][u'开盘价']:
        y[i]=1
    else:
        y[i]=0          

'\ndata.shape[0]-dayfeature 是 4602 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4602, 751)\ndayfeature 是 150 \n他用下一天，第 151 天，的「開盤」vs 當天的「收盤」來當作這天是否有漲。這點怪怪的，應該用「第 151 天」\n的「收盤」來跟「第 150 天」的「收盤」比較才對。\n'

# 這裡選用 classifier 

In [28]:
# SVM
from sklearn import svm
clf=svm.SVC(kernel='poly')  # so what is SVM? See "11-提交-监督学习 - 课程导学.pdf"
result = []

In [31]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier()
result = []

In [36]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
result = []

In [39]:
# Bayes
from sklearn.naive_bayes import GaussianNB
clf=GaussianNB()
result = []

In [42]:
for i in range(100):
    x_train, x_test, y_train, y_test = \
                cross_validation.train_test_split(x, y, test_size = 0.2)
    clf.fit(x_train, y_train)
    result.append(np.mean(y_test == clf.predict(x_test)))

GaussianNB(priors=None)

GaussianNB(priors=None)

GaussianNB(priors=None)

GaussianNB(priors=None)

GaussianNB(priors=None)

...

GaussianNB(priors=None)

GaussianNB(priors=None)

GaussianNB(priors=None)

GaussianNB(priors=None)

GaussianNB(priors=None)

In [43]:
print("svm classifier accuacy:")
print(result)

svm classifier accuacy:
[0.51465798045602607, 0.53745928338762217, 0.54723127035830621, 0.5233441910966341, 0.53745928338762217, 0.50488599348534202, 0.54940282301845822, 0.52008686210640609, 0.53203040173724214, 0.51574375678610207, 0.53528773072747016, 0.53311617806731815, 0.55917480998914226, 0.52985884907709013, 0.52442996742671011, 0.52985884907709013, 0.55483170466883824, 0.53203040173724214, 0.52660152008686212, 0.53311617806731815, 0.53637350705754616, 0.52985884907709013, 0.55591748099891425, 0.48751357220412594, 0.51248642779587406, 0.53854505971769817, 0.52551574375678611, 0.53094462540716614, 0.52660152008686212, 0.50488599348534202, 0.53311617806731815, 0.50705754614549403, 0.52985884907709013, 0.53637350705754616, 0.49728555917480999, 0.55700325732899025, 0.55157437567861023, 0.54831704668838221, 0.53745928338762217, 0.53203040173724214, 0.54180238870792619, 0.52117263843648209, 0.5222584147665581, 0.52985884907709013, 0.53311617806731815, 0.51465798045602607, 0.532030401

    --------------- play ground ---------------------

In [ ]:
x_train, x_test, y_train, y_test = \
                cross_validation.train_test_split(x, y, test_size = 0.2)

In [ ]:
%f cr ."  ----------- x_train ------------ " cr
%f __main__ :> x_train . cr
%f __main__ :> x_train type . cr
%f __main__ :> x_train.shape . ."  <-- 這麼多筆的資料，每筆有 150 天，每天 5 個行情數，共 750 個 feature 加 1 個 label " cr
%f __main__ :> x_train[0,0] . space 
%f __main__ :> x_train[0,1] . space
%f __main__ :> x_train[0,749] . space
%f __main__ :> x_train[0,750] . cr
%f cr ."  ----------- y_train ------------ " cr
%f __main__ :> y_train . ."  <-- 看看 y 的樣子 " cr
%f __main__ :> y_train type . cr
%f __main__ :> y_train.shape . cr

%f cr ."  ----------- x ------------ " cr
%f __main__ :> x . cr
%f __main__ :> x type . cr
%f __main__ :> x.shape . ."  <-- 這麼多筆的資料，每筆有 150 天，每天 5 個行情數，共 750 個 feature 加 1 個 label " cr
%f __main__ :> x[0,0] . space 
%f __main__ :> x[0,1] . space
%f __main__ :> x[0,749] . space
%f __main__ :> x[0,750] . cr

%f cr ."  ----------- y ------------ " cr
%f __main__ :> y . ."  <-- 看看 y 的樣子 " cr
%f __main__ :> y type . cr
%f __main__ :> y.shape . cr

In [ ]:
%f 3681 4602 / . cr

    ------------ 研究看漲跌停的趴數是多少？ -----------------

### Sort data['涨跌幅'] 即可知漲跌停趴數

e.g. `data.sort_index(0,ascending=True,inplace=True)` # 排序 sort 過後變成最舊的在上面

以下得知漲跌停幅度是： $\pm 10\%$ 

所謂「涨跌幅」是用「收盤價」來比的：$\frac{現價 - 前一天的收盤價}{前一天的收盤價} \times 100 \%$

In [ ]:
data.query('index>"1997-07-09" & index<"1997-07-16"')[['收盘价','最高价','最低价','开盘价','涨跌幅']]

In [12]:
ratio = data['涨跌幅'].sort_values()
%f cr ." 這樣看起來上證的漲停幅度是 10% 實際都不會真的頂到" cr
%f __main__ :> ratio :> [-120:-112] . cr cr 
%f ." 這樣看起來上證的跌停幅度是 -10% 實際都不會真的頂到" cr 
%f __main__ :> ratio :> [2195:2202] . cr 



這樣看起來上證的漲停幅度是 10% 實際都不會真的頂到
日期
2007-08-13    9.9936
2007-04-04     9.995
2015-09-08    9.9951
2010-10-13     9.996
2015-09-24    9.9964
2010-11-01    9.9971
2015-04-16    9.9975
2005-12-16      None
Name: 涨跌幅, dtype: object

這樣看起來上證的跌停幅度是 -10% 實際都不會真的頂到
日期
2015-06-19    -9.9953
2015-08-25    -9.9961
2016-01-11    -9.9961
2008-03-17    -9.9969
2015-06-29     -9.997
2004-08-23        0.0
2007-02-26        0.0
Name: 涨跌幅, dtype: object


In [15]:
data.query('index < "2007-08-15" & index >= "2007-08-10" ')

,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,换手率,成交量,成交金额,总市值,流通市值
日期,,,,,,,,,,,,,,
2007-08-10,'000777,中核科技,31.22,31.22,29.60,29.60,28.38,2.84,10.007,27.5827,24466990,7.528785e+08,5.244960e+09,2.769339e+09
2007-08-13,'000777,中核科技,34.34,34.34,31.68,32.80,31.22,3.12,9.9936,18.5287,16435733,5.564775e+08,5.769120e+09,3.046095e+09
2007-08-14,'000777,中核科技,32.42,35.49,32.33,35.48,34.34,-1.92,-5.5911,18.9427,16802963,5.593244e+08,5.446560e+09,2.875784e+09


In [19]:
data.query('index == "2007-08-13"')['收盘价'] 
data.query('index == "2007-08-10"')['收盘价']

日期
2007-08-13    34.34
Name: 收盘价, dtype: float64

日期
2007-08-10    31.22
Name: 收盘价, dtype: float64

In [20]:
(34.34-31.22)/31.22 

0.09993593850096107

    ----------- Trace 看他怎麼製作 x feature ---------------
   

In [11]:
# 結論是： 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

%pdb on

for i in range(0,data.shape[0]-dayfeature):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))  # 第 0～149 天的資料，共 150 天 750 個數字
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']  # 第 150 天的開盤，一個數字
    assert(False)
    break

'''
ipdb> type(x)
<class 'numpy.ndarray'>

data.shape is (4752, 14) 每次抓 150 天的資料當作是一筆以便讓 AI 一次看 150 天，讓它觀察其中的端倪
所以 featurenum 是 750
第一筆 data[0:150] 不含第 150 天
data[0:150][[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']] 挑出有關的欄位 shape (150,5)

這兩行是一樣的，用 -1 比較聰明，讓電腦自動決定；用 featurenum 可收檢查的效果，發現潛在問題
np.array(data[i:i+dayfeature][[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum)).shape --> (1,750)
np.array(data[i:i+dayfeature][[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,-1).shape --> (1,750)

x.shape
(4602, 751)   以上 750個數字 加上 1 個數字，共 751 個數字。 給 AI 的一筆輸入資料。

i 最後到 4601 為止，4601 + 150 = 4751 確實是最後一天。
這天的 5 個參數沒有進 x, 只有「開盤價」進了最後一筆 x 的第 751 個 feature.

'''    

Automatic pdb calling has been turned ON


AssertionError: 

> <ipython-input-11-da62440bc506>(8)<module>()
      6     x[i,0:featurenum]=np.array(data[i:i+dayfeature]           [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))  # 第 0～149 天的資料，共 150 天 750 個數字
      7     x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']  # 第 150 天的開盤，一個數字
----> 8     assert(False)
      9     break
     10 

ipdb> type(x)
<class 'numpy.ndarray'>
ipdb> c


In [10]:

%pdb on
for i in range(0,data.shape[0]-dayfeature):
    if data.iloc[i+dayfeature][u'收盘价']>=data.iloc[i+dayfeature][u'开盘价']:
        y[i]=1
    else:
        y[i]=0          
    assert(False)
    break

'''
data.shape[0]-dayfeature 是 4602 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4602, 751)
dayfeature 是 150 
他用下一天，第 151 天，的「開盤」vs 當天的「收盤」來當作這天是否有漲。這點怪怪的，應該用「第 151 天」
的「收盤」來跟「第 150 天」的「收盤」比較才對。

i 最後到 4601 為止，4601 + 150 = 4751 確實是最後一天
'''

Automatic pdb calling has been turned ON


AssertionError: 

> <ipython-input-10-4a8199313993>(9)<module>()
      7     else:
      8         y[i]=0
----> 9     assert(False)
     10     break
     11 

ipdb> data.shape[0]-dayfeature
4602
ipdb> dayfeature
150
ipdb> c


    -------- 看懂 data.ix[i] 或 data.iloc[i] -----------------------
    其實沒什麼，就是 data[i] 不行，所以要 data.iloc[i] 

In [ ]:
print(3+dayfeature)
%f __main__ :> data :> query('index>"1997-07-09"&index<"1997-07-16"')[['收盘价','最高价','最低价','开盘价','涨跌幅']].iloc[3][u'开盘价'] . cr
%f __main__ :> data :> query('index>"1997-07-09"&index<"1997-07-16"')[['收盘价','最高价','最低价','开盘价','涨跌幅']].ix[3][u'开盘价'] . cr        

%f cr cr ." 以下這三個都一樣意思，就是取 data-frame 的哪幾 row " cr
data.ix[:2]
data.iloc[:2]     
data[:2]        # 這樣用與 ix() iloc() 無異

%f cr cr ." 但是 data[1] 就不行了！！必須透過 data.iloc[1] " cr
data.iloc[1]    
type(data.iloc) # pandas.core.indexing._iLocIndexer 特殊型態